# Programmazione imperativa in OCaml (cenni)

In questo capitolo saranno illustrate **XXXXXXXXXXXXXXXXXXXX** del linguaggio OCaml, in particolare relative al paradigma di programmazione funzionale, che quello su cui OCaml si basa principalmente. Il linguaggio prevede comunque aspetti di programmazione imperativa e a oggetti che vedremo successivamente.

### Riferimenti

Questo capitolo si basa su materiale ed esempi di codice tratti da:

- <A href="https://dev.realworldocaml.org/">Real World OCaml</A> (Versione 2), di Yaron Minsky, Anil Madhavapeddy e Jason Hickey

La licenza per l'utilizzo del testo e degli esempi di codice è disponibile nel sito indicato.

Inoltre, alcuni aspetti del linguaggio possono essere approfonditi consultando il manuale ufficiale di OCaml, disponibile a questo indirizzo:

- <A href="https://ocaml.org/manual/index.html">https://ocaml.org/manual/index.html</A>

## Introduzione

Fino ad ora abbiamo scritto codice secondo il paradigma funzionale: 

Abbiamo principalmente definito *funzioni pure*

- Calcolano il risultato sulla base dei propri parametri e delle variabili disponibili nella chiusura

e variabili/strutture dati *immutabili*

- Non è possibile modificarle assegnando nuovi valori

Nella *programmazione imperativa* invece:

* le variabili sono *mutabili* (si possono assegnare a piacimento)
* le funzioni causano *side effect* (possono modificare l'ambiente globale)

Esempi di side effects:

* lettura/assegnamento di una varibile globale (o non locale)
* stampa di un messaggio di output / lettura di un input
* sollevamento di una eccezione 

Tutto questo rende le funzioni *impure*

* il loro effetto non è più solo calcolare un risultato
* il risultato può essere influenzato da fattori esterni (variabili globali/input/...)


OCaml è concepito principalmente per la programmazione funzionale...

* .. ma include anche tutti i costrutti imperativi principali

Ne accenneremo brevemente alcuni:

* array e record mutabili
* riferimenti
* cicli
* eccezioni

Inoltre, OCaml prevede la possibilità di definire *classi e oggetti* in pieno stile object-oriented

* questo per il momento non lo vediamo

## Array

Gli array in OCaml:

- contengono tutti elementi dello stesso tipo
- hanno dimensione fissata al momento della creazione  (come in C, Java, ...)
- possono essere acceduti tramite indici numerici (non sono associativi)

Definizione di un nuovo array:
   
- letterale (elencandone gli elementi tra `[|` e `|]`

In [1]:
let a = [|3;5;2|] ;;

val a : int array = [|3; 5; 2|]


- tramite `Array.make` (con parametri la dimensione e il valore iniziale degli elementi)

In [2]:
Array.make 10 0;;

- : int array = [|0; 0; 0; 0; 0; 0; 0; 0; 0; 0|]


La lunghezza di un array può essere ottenuta tramite `Array.lenght`

In [3]:
let n = Array.length a;;

val n : int = 3


Accedere agli elementi di un array (con indici in `0,...,(n-1)`):

- in lettura, con la sintassi `.(i)`

In [4]:
let e = a.(1) ;;

val e : int = 5


- in scrittura, con la sintassi `<-`

In [5]:
a.(1) <- 6 ;;
a;;

- : unit = ()


- : int array = [|3; 6; 2|]


### Un commento su `<-`

Il costrutto `<-` è un *COMANDO di ASSEGNAMENTO*

- è la prima volta che *modifichiamo* qualcosa!

Non è l'unico costrutto di assegnamento

- ne vedremo altri con sintassi diversa

Il *tipo dei comandi* in OCaml è `unit`

- in realtà non sono comandi, ma *espressioni con side effect*
- simile a `void` in altri linguaggi di programmazione
- descrive le cose che non possono essere valutate a valori
- l'unico valore del tipo `unit` è `()`


## Record mutabili

E' possibile rendere mutabili uno o più campi tramite il modificatore `mutable`

In [6]:
type persona = 
{
    nome: string;
    cognome: string;
    mutable eta: int;
}

type persona = { nome : string; cognome : string; mutable eta : int; }


In [7]:
let mario = {nome="mario"; cognome="rossi"; eta=30 } ;;

val mario : persona = {nome = "mario"; cognome = "rossi"; eta = 30}


Anche in questo caso per l'assegnamento si usa `<-`

In [8]:
mario.eta <- 31 ;;
mario ;;

- : unit = ()


- : persona = {nome = "mario"; cognome = "rossi"; eta = 31}


In [9]:
let invecchia p =
    p.eta <- p.eta + 1;;
    
invecchia mario;;
mario ;;

val invecchia : persona -> unit = <fun>


- : unit = ()


- : persona = {nome = "mario"; cognome = "rossi"; eta = 32}


## Riferimenti (refs)

Oltre ad array e record mutabili, è possibile definire anche singole variabili mutabili usando il tipo `ref`

Ad esempio:

In [10]:
let x = ref 12 ;;

val x : int ref = {contents = 12}


Si definisce la variabile `x` che contiene un *riferimento*

* il riferimento punta a un record con un solo campo mutabile `contents` inizializato con il valore passato

E' possibile accedere alla variabile `ref` come record

In [11]:
x.contents ;;
x.contents <- 13;;

- : int = 12


- : unit = ()


Ma in realtà c'è una *sintassi specifica*, più semplice:

In [12]:
!x ;;            (* corrisponde a x.contents *)
x := 14 ;;       (* corrisponde a x.contents <- 14 *)

- : int = 13


- : unit = ()


L'operazione `!` è detta *dereferenziazione*

Con `ref` si creano effettivamente dei riferimenti

* dei puntatori ad aree di memoria

Copiando il riferimento in una nuova variabile, entrambe le variabili punteranno alla stessa area di memoria

**TODO: FARE FIGURA**

In [42]:
let x = ref 0;;
let y = x;;
y:=100;;
!x;;

val x : int ref = {contents = 0}


val y : int ref = {contents = 0}


- : unit = ()


- : int = 100


Modificando `y` anche `x` risulta modificata

## Sequenze di comandi e cicli

Una sequenza di comandi/espressioni può essere definita tramite il separatore `;`

In [43]:
let x = ref 0;;
let y = x;;
x := !x+1 ; y := !y+1 ; (!x,!y) ;; (* sequenza di espressioni *)

val x : int ref = {contents = 0}


val y : int ref = {contents = 0}


- : int * int = (2, 2)


Le espressioni vengono valutate una dopo l'altra e l'ultima fornisce il risultato finale

In una sequenza di espressioni `E1 ; E2 ; .... ; En`:

- tutte le espressioni precedenti all'ultima devono (dovrebbero...) avere tipo `unit`
- le espressioni `E1 ; E2 ; .... ; E(n-1)` causano solo side effects (ad esempio, modificando le strutture dati mutabili)
- l'espressione `En` calcola il risultato vero e proprio

In [64]:
x := !x+1 ; y := !y+1 ; (!x,!y) ;;

- : int * int = (4, 4)


In [81]:
4+3 ; 5+2 ;;

File "[81]", line 1, characters 0-3:
1 | 4+3 ; 5+2 ;;
    ^^^
Warning 10: this expression should have type unit.
File "[81]", line 1, characters 0-3:
1 | 4+3 ; 5+2 ;;
    ^^^
Warning 10: this expression should have type unit.


- : int = 7


### NOTA: `;` vs `in`

Viene la tentazione di usare `;` per separare dichiarazioni di variabili dal loro uso...

In [89]:
let f n = n+1 ;; (* funzione giocattolo, giusto per l'esempio *)

let m = 10 ; f(m) ;;

val f : int -> int = <fun>


File "[89]", line 3, characters 8-10:
3 | let m = 10 ; f(m) ;;
            ^^
Warning 10: this expression should have type unit.


error: compile_error

invece bisogna usare `in`

In [90]:
let m = 10 in f(m) ;;

- : int = 11


In [68]:
let media arr =
    let s = ref 0 in
    for i = 0 to (Array.length arr)-1 do
        s := !s + arr.(i)
    done ;
    !s / Array.length arr ;;
    
media [|4;6;8|] ;;

val media : int array -> int = <fun>


- : int = 6


## Eccezioni

In [13]:
failwith ;;

- : string -> 'a = <fun>


In [14]:
failwith "aaa";;

error: runtime_error

In [15]:
assert ;;

error: compile_error

In [16]:
raise ;;

- : exn -> 'a = <fun>


In [17]:
printf ;;

error: compile_error

In [18]:
print_float 3.;
print_endline "";;

3.


- : unit = ()


In [19]:
print_string "CCC";
flush stdout ;;

- : unit = ()


In [20]:
print_newline (print_string "ciao");;

CCCciao


- : unit = ()
